In [ ]:
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

# kaggle 데이터 불러오기
import http.client

# 그래프 그리기 
import matplotlib.pyplot as plt
import seaborn as sns

# crawling
from bs4 import BeautifulSoup
import urllib.request as req
import time

import missingno as msno

-----
### Kaggle


In [ ]:
# !pip install kagglehub

In [ ]:


conn = http.client.HTTPSConnection("travel-advisor.p.rapidapi.com")

payload = "{\"contentType\":\"hotel\",\"contentId\":\"4172546\",\"questionId\":\"8393250\",\"pagee\":0,\"updateToken\":\"\"}"

headers = {
    'x-rapidapi-key': "aec43032d7msh6f891ac493424f0p19965ajsnfb40027abe1b",
    'x-rapidapi-host': "travel-advisor.p.rapidapi.com",
    'Content-Type': "application/json"
}

conn.request("POST", "/answers/v2/list?currency=USD&units=km&lang=en_US", payload, headers)

res = conn.getresponse()
data = res.read()
data = data.decode('utf-8')

In [ ]:
print(data)

---------
### Crawling

#### 한강공원 이용자 통계
- 컬럼 CSS 선택자 : '#anchor-content > div.contents-area > div > div.usage-status-table > div:nth-child(2) > table > thead > tr:nth-child(2)
- 공원별 id : AID(4~14)


In [ ]:
year = list(range(2018,2025)) # url page 년도 
year
month = list((range(1,13))) # url page 월

In [ ]:
monthstr = [] # url 형식에 맞게 변경
for m in month :
    # print('0'+f'{m}')
    if m < 10:
        m = '0' + f'{m}'
        monthstr.append(m)
    else:
        monthstr.append(f'{m}')

print(monthstr)

> 202403 - 202409 월별 한강 이용객 통계 크롤링 

In [ ]:
# 신 한강공원 
# 한강공원 이용객통계(공원별) - 전체 이용객, 일평균 이용객, 현지인 이용객, 외지인 이용객,	외국인 이용객, 자전거 이용객 등 합계통계
new_result = []
for month in range(3,10):
        url = f'https://hangang.seoul.go.kr/www/utztnStatsNew/utztnStats.do?mid=904&opt2=2024-{month}&opt3=2024-{month+1}&sdtYear=2024&sdtMonth={month}&edtYear=2024&edtMonth={month}'
        try:
            res = req.urlopen(url)
            time.sleep(5)
            soup = BeautifulSoup(res, 'html.parser')
            
            month_data = []
            for i in range(1, 12):
                row_data = []
                for td in soup.select(f'#AID{i} > td'):
                    row_data.append(td.get_text().strip())
                month_data.append(row_data)
            
            new_result.append(month_data)
        except Exception as e:
            print(f'Error occurred for {month}: {str(e)}')
        time.sleep(4)

print(f"Total months of data collected: {len(new_result)}")


In [ ]:
new_df = pd.concat([pd.DataFrame(month_data) for month_data in new_result], ignore_index=True)

# 컬럼 이름 설정

# 결과 확인
print(new_df.head())
print(f"Total rows: {len(new_df)}")


In [ ]:
new_df.columns=['전체 이용객','일평균 이용객','현지인 이용객','외지인 이용객','외국인 이용객','자전거 이용객','선호 연령대','선호 시간대']

In [ ]:
new_df['지점'] = ['광나루','잠실','뚝섬','잠원','반포','이촌','여의도','양화','난지','망원','강서']*7

In [ ]:
new_df.to_csv('../../Data/한강2403-2409.csv',index=False)

> 201801 - 202402 한강 월별 이용객 통계 크롤링

In [ ]:
# 구 한강공원 데이터 크롤링
import urllib.request as req
from bs4 import BeautifulSoup
import time

result = []
years = range(2018, 2025)  # 2018부터 2024까지
months = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

for year in years:
    for month in months:
        url = f'https://hangang.seoul.go.kr/www/utztnStats/utztnStats.do?mid=622&opt2={year}-{month}&opt3={year}-{month}&sdtYear={year}&sdtMonth={month}&edtYear={year}&edtMonth={month}'
        
        try:
            res = req.urlopen(url)
            time.sleep(4)
            soup = BeautifulSoup(res, 'html.parser')
            
            month_data = []
            for i in range(4, 15):
                row_data = []
                for td in soup.select(f'#AID{i} > td'):
                    row_data.append(td.get_text().strip())
                month_data.append(row_data)
            
            result.append(month_data)
        except Exception as e:
            print(f'Error occurred for {year}-{month}: {str(e)}')
        if year == 2024 and month == '02':
            break
        time.sleep(4)

print(f"Total months of data collected: {len(result)}")


In [ ]:

# 모든 월별 데이터를 하나의 데이터프레임으로 결합
df = pd.concat([pd.DataFrame(month_data) for month_data in result], ignore_index=True)

# 컬럼 이름 설정

# 결과 확인
print(df.head())
print(f"Total rows: {len(df)}")


----
## 주차장일별


> 지구별 코드 변경

In [ ]:
주차장일별 = pd.read_csv('../../Data/한강공원 주차장 일별 이용 현황.csv', encoding='euc-kr')

In [ ]:
주차장일별

In [ ]:
# 주차장일별['code'] 
codelist = []
for code in 주차장일별['지구별']:
    codelist.append(code.replace('PLT-',""))

In [ ]:
주차장일별['code'] = codelist

> 날짜를 년,월,일로 구분

In [ ]:
주차장일별['날짜']= pd.to_datetime(주차장일별['날짜'], format='%Y/%m/%d')

In [ ]:
주차장일별['년도']= 주차장일별['날짜'].dt.year
주차장일별['월']= 주차장일별['날짜'].dt.month
주차장일별['일']= 주차장일별['날짜'].dt.day

In [ ]:
주차장일별

In [ ]:
%pip install holidays

In [ ]:
# 평일 휴일로 분리 시켜주기 
import holidays

# 대한민국 공휴일 가져오기
kr_holidays = holidays.KR(years=range(2020,2025))

# 날짜가 공휴일인지를 체크하는 함수 추가
주차장일별['휴일여부'] = 주차장일별['날짜'].apply(lambda x: '휴일' if x in kr_holidays else ('주말' if x.weekday() >= 5 else '평일'))


In [ ]:
주차장일별.info()

In [ ]:
주차장일별

In [ ]:
주차장일별['이용시간'] = (주차장일별['이용시간']/60) / 주차장일별['주차대수']

In [ ]:
주차장일별.info()

-----
### 날씨

> 날씨 강수량, 강수지속시간 null값 처리

In [ ]:
한강날씨 = pd.read_csv('../../Data/서울날씨20-24.csv',encoding='euc-kr')

In [ ]:
한강날씨['강수 계속시간(hr)'].fillna(0,inplace=True)
한강날씨['일강수량(mm)'].fillna(0,inplace=True)

In [ ]:
# 20220808 서울 날씨
# 최저기온 24도, 최저기온시각 13시, 14시 
# 검색을 통해서 얻은 정보
한강날씨['최저기온(°C)'].fillna(24,inplace=True)
한강날씨['최저기온 시각(hhmi)'].fillna(1300,inplace=True)

In [ ]:
한강날씨.info()

-------
### 서울시 한강공원 월별 이용객


In [ ]:
월별이용객통계=pd.read_csv('../../Data/서울시 한강공원 월별 이용객 현황.csv', encoding='euc-kr')

In [ ]:
월별이용객통계

In [ ]:
주차장일별

----------
### 날씨, 월별 주차장 병합


In [ ]:
한강날씨['일시'] = pd.to_datetime(한강날씨['일시'])
한강날씨.info()

In [ ]:
한강날씨['일시'].value_counts()

In [ ]:
merge_weather= pd.merge(주차장일별, 한강날씨, how= 'left', right_on='일시', left_on='날짜')

In [ ]:
merge_weather.info()

In [ ]:
merge_weather.drop('지점', axis =1 , inplace=True)

In [ ]:

# 상관계수 계산
correlation_matrix = merge_weather.select_dtypes('number').corr()

# 그림 크기 설정
plt.figure(figsize=(10, 8))

# heatmap 생성
sns.heatmap(correlation_matrix, 
            annot=True,  # 각 셀에 숫자 표시
            cmap='coolwarm',  # 색상 맵 설정
            vmin=-1, vmax=1,  # 색상 범위 설정
            center=0,  # 0을 중심으로 색상 대칭
            square=True,  # 정사각형 셀
            fmt='.2f')  # 소수점 둘째 자리까지 표시

# 제목 설정
plt.title('Correlation Heatmap', fontsize=16)

# 그래프 표시
plt.show()


In [ ]:
# 
merge_weather.head()

In [ ]:
merge_weather.to_csv('../../Data/날씨+일별주차이용.csv',index=False)

----
## 주차장 정보

In [ ]:
주차장정보 = pd.read_csv('../../Data/한강공원 주차장 정보.csv',encoding='euc-kr')

In [ ]:
print(주차장정보['주차장별'].sort_values().unique())
print(주차장일별['주차장명'].sort_values().unique())
# 주차장 정보 : 
# 주차장 일별 : 

In [ ]:
월별이용객통계
hzone_mapping = {
    'Hzone011': '강서',
    'Hzone009': '난지',
    'Hzone010': '망원',
    'Hzone006': '이촌',
    'Hzone003': '뚝섬',
    'Hzone001': '광나루',
    'Hzone008': '양화',
    'Hzone007': '여의도',
    'Hzone005': '반포',
    'Hzone004': '잠원',
    'Hzone002': '잠실'
}
월별이용객통계['이름'] = 월별이용객통계['공원 코드'].map(hzone_mapping)


In [ ]:
월별이용객통계

------

In [ ]:
print(merge_weather['주차장명'].unique())
(주차장정보['주차장별'].sort_values().unique())

In [ ]:
월별이용객통계= 월별이용객통계.loc[:,['이름','일반이용자(아침)','일반이용자(낮)','일반이용자(저녁)','주요행사','현황 일시','등록 일시']]

In [ ]:
new_df.rename(columns={'지점' : '이름'}, inplace= True)
print(월별이용객통계.columns)

In [ ]:
new_df.drop(['외지인 이용객','외국인 이용객'],axis = 1, inplace = True) 

In [ ]:
new_df.drop(['자전거 이용객','선호 연령대','선호 시간대'], axis = 1, inplace=True )

In [ ]:
월별이용객통계.drop('등록 일시',axis = 1, inplace= True)

In [ ]:
월별이용객통계['일반이용자 합']= 월별이용객통계['일반이용자(아침)'] + 월별이용객통계['일반이용자(낮)'] + 월별이용객통계['일반이용자(저녁)']

In [ ]:
월별이용객통계['아침']= 월별이용객통계['일반이용자(아침)']/ 월별이용객통계['일반이용자 합'] 
월별이용객통계['낮']= 월별이용객통계['일반이용자(낮)']/ 월별이용객통계['일반이용자 합'] 
월별이용객통계['저녁']= 월별이용객통계['일반이용자(저녁)']/ 월별이용객통계['일반이용자 합'] 

In [ ]:
월별이용객통계

In [ ]:
merge_weather

In [ ]:
from sklearn.model_selection import train_test_split, cross_validate
import numpy as np